# Ejemplo clásico de EM y su aplicación para encontrar los parámetros de GMM.

Supongamos que tenemos dos monedas A y B, cada una con probabilidad $\theta_A$ y $\theta_B$ respectivamente de que salga cara. No sabemos cual es cual pero queremos estimar $\theta_A$ y $\theta_B$ utilizando este procedimiento: 

* Escogemos una de las dos monedas al azar y la tiramos 10 veces, anotando los resultados $x_i=\{H,H,H,T,T,T,...\}$ con $ 1 \leq i \leq 5 $ y la moneda que escogimos $z_i=\{A|B\}$
* Repetimos el punto anterior 5 veces.

En este caso podemos calcular $\theta_A$ y $\theta_B$ como:

$$\theta_A=\frac{\text{# de caras que salieron de A}}{\text{total de # que tiré la moneda A}} $$

$$\theta_B=\frac{\text{# de caras que salieron de B}}{\text{total de # que tiré la moneda B}} $$

Supongamos ahora que no tenemos toda la información del experimento y solo tenemos los resultados $x_i$ pero no conocemos $z_i$. En el caso de un modelo GMM esto es así ya que no se a qué gaussiana del modelo corresponde la salida generada. En este caso decimos que z son variables ocultas.

Obviamente calcular la proporción de caras y secas para esta caso no tiene sentido ya que no se a qué moneda atribuirle cada una de las instancias i del experimento.

Si aplicamos EM para este problema los pasos serían:

1) Asignar valores iniciales a los parámetros $\theta_A(0)$ y $\theta_B(0)$.  
2) Estimamos para cada instancia del experimento qué moneda es mas probable que haya generado el resultado. Para el caso de la moneda sabemos que tiene distribución binomial de parámetro $\theta_A(t)$ y $\theta_B(t)$ y $N=10$. Podemos eliminar la combinatoria de la distribución binomial ya que solo nos interesa cual es la mayor.  
3) Con el likelihood para A y B del punto anterior, podemos calcular la probabilidad de que se haya utilizado cada una de las monedas, normalizando cada likelihood por la suma de ellos.  

4) Utilizamos una versión modificada del Maximun Likelihood la cual pesa a cada uno de los resultados obtenidos en cada instancia del experimento por la probabilidad de que se haya utilizado cada moneda, y en base a eso se calculan los nuevos parámetros $\theta_A(t+1)$ y $\theta_B(t+1)$. Para mayor claridad se puede ver el código a continuación o bien este [paper](http://ai.stanford.edu/~chuongdo/papers/em_tutorial.pdf)

5) Ir al paso 2 hasta que converja

In [188]:
import numpy as np
#Los resultados de los experimentos son:
x=np.array([[1,0,0,0,1,1,0,1,0,1],[1,1,1,1,0,1,1,1,1,1],[1,0,1,1,1,1,1,0,1,1],[1,0,1,0,0,0,0,0,1,1],
   [0,1,1,1,0,1,1,1,0,1]])

# Y los valores de tetha A y B iniciales son:
theta_A=0.5
theta_B=0.6

print("Expectation:")
# Estimamos para cada iteración qué moneda fue mas probable utilizando el maximum likelihood:
likelihood_A=np.zeros(5)
likelihood_A=theta_A**x.sum(axis=1)*(1-theta_A)**(x.shape[1]-x.sum(axis=1))
print("El likelihood de A para cada instancia del experimento es:")
print(likelihood_A)

likelihood_B=np.zeros(5)
likelihood_B=theta_B**x.sum(axis=1)*(1-theta_B)**(x.shape[1]-x.sum(axis=1))
print("El likelihood de A para cada instancia del experimento es:")
print(likelihood_B)

prob_A=likelihood_A/(likelihood_A+likelihood_B)
print("La probabilidad de que haya sido A la moneda que se tiró en cada instancia es:")
print(prob_A)

prob_B=likelihood_B/(likelihood_A+likelihood_B)
print("La probabilidad de que haya sido A la moneda que se tiró en cada instancia es:")
print(prob_B)

print("Maximization:")
#Doy por sentado que el resultado del ML es correcto, y por lo tanto tengo los datos completos
caras_a=0
caras_b=0
secas_a=0
secas_b=0
for i in range(5):
    caras_a=caras_a+prob_A[i]*x[i].sum()
    secas_a=secas_a+prob_A[i]*(x[i].size-x[i].sum())
    caras_b=caras_b+prob_B[i]*x[i].sum()
    secas_b=secas_b+prob_B[i]*(x[i].size-x[i].sum())
    print(str(caras_a)+" "+str(secas_a)+" "+str(caras_b)+" "+str(secas_b))    
#Estimo los nuevos valores de theta_A y theta_B
theta_A=caras_a/(caras_a+secas_a)
theta_B=caras_b/(caras_b+secas_b)
print("El nuevo valor de theta A es:")
print(theta_A)
print("El nuevo valor de theta B es:")
print(theta_B)

Expectation:
El likelihood de A para cada instancia del experimento es:
[ 0.00097656  0.00097656  0.00097656  0.00097656  0.00097656]
El likelihood de A para cada instancia del experimento es:
[ 0.00079626  0.00403108  0.00268739  0.00053084  0.00179159]
La probabilidad de que haya sido A la moneda que se tiró en cada instancia es:
[ 0.55085107  0.19501448  0.26653284  0.64784387  0.35278488]
La probabilidad de que haya sido A la moneda que se tiró en cada instancia es:
[ 0.44914893  0.80498552  0.73346716  0.35215613  0.64721512]
Maximization:
2.7542553695 2.7542553695 2.2457446305 2.2457446305
4.5093857144 2.94926985226 9.4906142856 3.05073014774
6.64164845033 3.48233553624 15.3583515497 4.51766446376
9.23302391495 7.36939873317 16.7669760851 6.63060126683
11.7025181037 8.42775338547 21.2974818963 8.57224661453
El nuevo valor de theta A es:
0.581339308314
El nuevo valor de theta B es:
0.713012235401


Realicemos mas iteraciones:

In [189]:
for i in range(10):
    print("Expectation:")
    # Estimamos para cada iteración qué moneda fue mas probable utilizando el maximum likelihood:
    likelihood_A=np.zeros(5)
    likelihood_A=theta_A**x.sum(axis=1)*(1-theta_A)**(x.shape[1]-x.sum(axis=1))
    print("El likelihood de A para cada instancia del experimento es:")
    print(likelihood_A)

    likelihood_B=np.zeros(5)
    likelihood_B=theta_B**x.sum(axis=1)*(1-theta_B)**(x.shape[1]-x.sum(axis=1))
    print("El likelihood de A para cada instancia del experimento es:")
    print(likelihood_B)

    prob_A=likelihood_A/(likelihood_A+likelihood_B)
    print("La probabilidad de que haya sido A la moneda que se tiró en cada instancia es:")
    print(prob_A)

    prob_B=likelihood_B/(likelihood_A+likelihood_B)
    print("La probabilidad de que haya sido A la moneda que se tiró en cada instancia es:")
    print(prob_B)

    print("Maximization:")
    #Doy por sentado que el resultado del ML es correcto, y por lo tanto tengo los datos completos
    caras_a=0
    caras_b=0
    secas_a=0
    secas_b=0
    for i in range(5):
        caras_a=caras_a+prob_A[i]*x[i].sum()
        secas_a=secas_a+prob_A[i]*(x[i].size-x[i].sum())
        caras_b=caras_b+prob_B[i]*x[i].sum()
        secas_b=secas_b+prob_B[i]*(x[i].size-x[i].sum())
        print(str(caras_a)+" "+str(secas_a)+" "+str(caras_b)+" "+str(secas_b))    
    #Estimo los nuevos valores de theta_A y theta_B
    theta_A=caras_a/(caras_a+secas_a)
    theta_B=caras_b/(caras_b+secas_b)
    print("El nuevo valor de theta A es:")
    print(theta_A)
    print("El nuevo valor de theta B es:")
    print(theta_B)

Expectation:
El likelihood de A para cada instancia del experimento es:
[ 0.000854    0.0031749   0.00228645  0.00061502  0.00164662]
El likelihood de A para cada instancia del experimento es:
[ 0.00035876  0.01366898  0.00550177  0.0001444   0.00221447]
La probabilidad de que haya sido A la moneda que se tiró en cada instancia es:
[ 0.70418068  0.18848955  0.29357799  0.80985546  0.42646607]
La probabilidad de que haya sido A la moneda que se tiró en cada instancia es:
[ 0.29581932  0.81151045  0.70642201  0.19014454  0.57353393]
Maximization:
3.52090337618 3.52090337618 1.47909662382 1.47909662382
5.21730935156 3.709392929 8.78269064844 2.290607071
7.56593325759 4.29654890551 14.4340667424 3.70345109449
10.8053551031 9.15568167379 15.1946448969 4.84431832621
13.7906175799 10.4350798781 19.2093824201 6.56492012186
El nuevo valor de theta A es:
0.569255750172
El nuevo valor de theta B es:
0.745292036082
Expectation:
El likelihood de A para cada instancia del experimento es:
[ 0.0008864

In [177]:
x=np.array([[1,0,0,0,1,1,0,1,0,1],[1,1,1,1,0,1,1,1,1,1],[1,0,1,1,1,1,1,0,1,1],[1,0,1,0,0,0,0,0,1,1],
   [0,1,1,1,0,1,1,1,0,1]])
theta_A=0.6
theta_B=0.5

In [180]:
likelihood_A=theta_A**x.sum(axis=1)+(1-theta_A)**([10,10,10,10,10]-x.sum(axis=1))
print(likelihood_A)
likelihood_B=theta_B**x.sum(axis=1)+(1-theta_B)**([10,10,10,10,10]-x.sum(axis=1))
print(likelihood_B)

[ 0.088       0.4100777   0.17679616  0.133696    0.0919936 ]
[ 0.0625      0.50195312  0.25390625  0.078125    0.1328125 ]


In [175]:
theta_A**x.sum(axis=1)

array([ 0.15661464,  0.03553843,  0.05149105,  0.22691639,  0.07460454])

In [183]:
0.5**x.sum(axis=1)

array([ 0.03125   ,  0.00195312,  0.00390625,  0.0625    ,  0.0078125 ])

Se puede ver cómo aplicar EM a un modelo GMM [acá](http://aishack.in/tutorials/expectation-maximization-gaussian-mixture-model-mixtures/)